In [ ]:
from PIL import Image as pilimg
import numpy as np
import os
import time
import cv2
import torch
import torchvision
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.mobile_optimizer import optimize_for_mobile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
IMG_SIZE = 224
NUM_CLASSES= 14
IMAGE_SHAPE = [224,224]
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
train_loss_hist = []
val_loss_hist = []

# Defining Functions

In [ ]:
def load_images_from_folder(folder,i): #Couldn't use common image loaders due to diversity of extensions
    image = []
    image_y= []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if img is not None:
            img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
            img = img.reshape(IMG_SIZE,IMG_SIZE,3)
            image.append(img)
            image_y.append(i)
    return image,image_y

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=25):
   since = time.time()

   for epoch in range(num_epochs):
       print('Epoch {}/{}'.format(epoch, num_epochs - 1))
       print('-' * 10)

       #set model to trainable
       # model.train()

       train_loss = 0
       tot_val_loss = 0
       # Iterate over data.
       for i, data in enumerate(dataloaders['train']):
           inputs , labels = data
           inputs = inputs.to(device)
           labels = labels.to(device)
          
           optimizer.zero_grad()
          
           with torch.set_grad_enabled(True):
               outputs  = model(inputs)
               loss = criterion(outputs, labels)
           
               
           
           loss.backward() # calculates grads of all trainable parameter
           optimizer.step() #updates parameters

           train_loss += loss.item() * inputs.size(0)
           print('{} Loss: {:.4f} '.format(
               'train', train_loss / dataset_sizes['train']))
       train_loss_hist.append(train_loss / dataset_sizes['train'])
           
           
       for i, val_data in enumerate(dataloaders['validation']):
           val_inputs , val_labels = val_data
           val_inputs = val_inputs.to(device)
           val_labels = val_labels.to(device)
           optimizer.zero_grad()
           with torch.set_grad_enabled(True):
               val_outputs  = model(val_inputs)
               val_loss= criterion(val_outputs, val_labels)
           val_loss.backward()
           tot_val_loss += val_loss.item() * val_inputs.size(0)
       val_loss_hist.append(tot_val_loss / dataset_sizes['validation'])
       print('{} Loss: {:.4f} '.format('validation', tot_val_loss / dataset_sizes['validation']))
          
   time_elapsed = time.time() - since
   print('Training complete in {:.0f}m {:.0f}s'.format(
       time_elapsed // 60, time_elapsed % 60))
   lst_iter = range(num_epochs)
   plt.plot(lst_iter, train_loss_hist, '-b', label='train_loss')
   plt.plot(lst_iter, val_loss_hist, '-r', label='val_loss')
   plt.xlabel("n iteration")
   plt.legend(loc='upper left')

   return model

In [ ]:
def visualize_model(model, num_images=6):
   was_training = model.training
   model.eval()
   images_so_far = 0

   with torch.no_grad():
       for i, (inputs, labels) in enumerate(dataloaders['validation']):
           inputs = inputs.to(device)
           labels = labels.to(device)

           outputs = model(inputs)
           _, preds = torch.max(outputs, 1)

           for j in range(inputs.size()[0]):
               images_so_far += 1
               ax = plt.subplot(num_images//2, 2, images_so_far)
               ax.axis('off')
               ax.set_title('predicted: {} truth: {}'.format(class_names[preds[j]], class_names[labels[j]]))
               img = inputs.cpu().data[j].numpy().transpose((1, 2, 0))
               img = std * img + mean
               ax.imshow(img)

               if images_so_far == num_images:
                   model.train(mode=was_training)
                   return
       model.train(mode=was_training)

# Main Code

In [ ]:
labels = ["basking", "blacktip", "blue", "bull", "hammerhead", "lemon","mako", "nurse", "sandtiger", "thresher","tiger", "whale", "white", "whitetip"]
X = np.empty((0,IMG_SIZE,IMG_SIZE,3))
y = np.empty((0,1))
for i in range(len(labels)):
    images, images_y = load_images_from_folder(r"sharks/"+labels[i],i)
    X = np.append(X,images,0)
    y = np.append(y,images_y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
for i in range(len(X_train)): #Numpy arrays converted back to files to save in DataLoader as JPEG
    im = pilimg.fromarray(X_train[i].astype("uint8"))
    im.save(f'sharks_final/train/{labels[int(y_train[i])]}/{i}.png', "JPEG")
for i in range(len(X_test)):
    im = pilimg.fromarray(X_test[i].astype("uint8"))
    im.save(f'sharks_final/validation/{labels[int(y_test[i])]}/{i}.png', "JPEG")

In [ ]:
data_transforms = {
   'train': transforms.Compose([
       transforms.CenterCrop(IMG_SIZE),
       transforms.RandomHorizontalFlip(0.5), #augmentation
       transforms.RandomCrop(IMG_SIZE, pad_if_needed= True),
       transforms.RandomPerspective(p=0.5),
       transforms.ColorJitter(brightness=1, contrast=1, saturation=1),
       transforms.ToTensor(),
       transforms.Normalize(mean, std)
   ]),
   'validation': transforms.Compose([
       transforms.CenterCrop(IMG_SIZE),
       transforms.ToTensor(),
       transforms.Normalize(mean, std)
       ]),
    }
    
    image_datasets = {
       x: datasets.ImageFolder(
           "sharks_final/"+x,
           transform=data_transforms[x]
       )
       for x in ['train', 'validation']
    }
    
    dataloaders = {
       x: torch.utils.data.DataLoader(
           image_datasets[x], batch_size=16,
           shuffle=True, num_workers=4
       )
       for x in ['train', 'validation']
    }
    

In [ ]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation']}
    print(dataset_sizes)

In [ ]:
class_names = image_datasets['train'].classes
    print(class_names)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
mobilenet = torchvision.models.mobilenet_v3_small(pretrained=True)

In [ ]:
for param in  mobilenet.parameters():
        param.requires_grad = False

In [ ]:
number_infeatures = mobilenet.classifier[3].in_features
features = list(mobilenet.classifier.children())[:-1] # Removes last layer
features.extend([torch.nn.Linear(number_infeatures, len(class_names))])
mobilenet.classifier = torch.nn.Sequential(*features)

mobilenet = mobilenet.to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss() #defining loss function
optimizer_ft = optim.SGD(mobilenet.parameters(), lr=0.001, momentum=0.9) #defining optimizer
mobilenet = train_model(mobilenet, criterion, optimizer_ft, num_epochs=30) #training the model

visualize_model(mobilenet)

# Saving Model for Mobile Apllication

In [ ]:
quantized_model = torch.quantization.convert(mobilenet)
example = torch.rand(1,3,224,224)
scripted_model = torch.jit.trace(quantized_model, example)
opt_model = optimize_for_mobile(scripted_model)
opt_model._save_for_lite_interpreter("model_output/MobileNetV3.ptl")